<a href="https://colab.research.google.com/github/james330/NLPHW05/blob/main/NLPHW05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install opencc-python-reimplemented

     |████████████████████████████████| 484 kB 5.1 MB/s 
  Created wheel for opencc-python-reimplemented: filename=opencc_python_reimplemented-0.1.6-py2.py3-none-any.whl size=486150 sha256=99db97833f9d92539a74d69b12ed0b287c68035f052def9afbb4910ba7f63ea0
  Stored in directory: /root/.cache/pip/wheels/4e/e2/60/d062d260be08788bb389521544a8fc173de9a9a78d6a593344
Successfully built opencc-python-reimplemented


In [2]:
import json
from opencc import OpenCC
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam # - Works

In [92]:
# 將json格式，轉為python格式
# {'english':'xxxx' ,'chinese':'xxxxx'}
data = []
datas = []
file = open('./drive/MyDrive/Colab Notebooks/data/translation2019zh_train.json' ,'r')
for f in file:
  data.append(json.loads(f)) #就這樣將json 轉成python了

In [93]:
# 分割英文數據和中文數據
en_data = []
ch_data = []
for da in data:
  en_data.append(da['english'])
  ch_data.append('\t'+da['chinese']+'\n')

print('英文數據:\n', en_data[:3])
print('\n中文數據:\n', ch_data[:3])

英文數據:
 ['For greater sharpness, but with a slight increase in graininess, you can use a 1:1 dilution of this developer.', 'He calls the Green Book, his book of teachings, “the new gospel.', 'And the light breeze moves me to caress her long ear']

中文數據:
 ['\t为了更好的锐度，但是附带的会多一些颗粒度，可以使用这个显影剂的1：1稀释液。\n', '\t他还把宣扬自己思想的所谓《绿皮书》称作“新福音书”。\n', '\t微风推着我去爱抚它的长耳朵\n']


In [96]:
# 分別生成中英文字典，英文
i = 0
for ed in en_data:
  while i < 1000:
    en_vocab = set(''.join(ed))|en_vocab
    i+=1

id2en = list(en_vocab)
en2id = {c:i for i,c in enumerate(id2en)}
print('\n英文字典:\n', en2id)


英文字典:
 {'a': 0, 'd': 1, 'G': 2, 'B': 3, 'q': 4, 'w': 5, 'n': 6, 'm': 7, 's': 8, 'f': 9, 't': 10, 'k': 11, ' ': 12, ':': 13, '.': 14, 'H': 15, 'u': 16, 'r': 17, 'W': 18, 'y': 19, '“': 20, '1': 21, 'v': 22, 'h': 23, ',': 24, 'c': 25, 'F': 26, 'g': 27, 'i': 28, 'p': 29, 'b': 30, 'T': 31, 'l': 32, 'z': 33, 'o': 34, 'e': 35}


In [99]:
# 分別生成中英文字典，中文
i = 0
for cd in ch_data:
  while i < 1000000:
    ch_vocab = set(''.join(cd))|ch_vocab
    i+=1

id2ch = list(ch_vocab)
ch2id = {c:i for i,c in enumerate(id2ch)}
print('\n中文字典共計\n:', ch2id)


中文字典共計
: {'绿': 0, '自': 1, '鲜': 2, '微': 3, '液': 4, '推': 5, '稀': 6, '烈': 7, '“': 8, '1': 9, '扬': 10, '还': 11, '》': 12, '把': 13, '。': 14, '他': 15, '《': 16, '附': 17, '称': 18, '些': 19, '度': 20, '一': 21, '个': 22, '\n': 23, '是': 24, '先': 25, '血': 26, '多': 27, '”': 28, '了': 29, '们': 30, '风': 31, '谓': 32, '作': 33, '朵': 34, '用': 35, '耳': 36, '，': 37, '带': 38, '福': 39, '的': 40, '去': 41, '\t': 42, '剂': 43, '…': 44, '音': 45, '想': 46, '使': 47, '皮': 48, '好': 49, '显': 50, '思': 51, '流': 52, '释': 53, '着': 54, '以': 55, '我': 56, '可': 57, '这': 58, '长': 59, '为': 60, '新': 61, '白': 62, '锐': 63, '但': 64, '书': 65, '宣': 66, '己': 67, '抚': 68, '影': 69, '爱': 70, '颗': 71, '会': 72, '所': 73, '更': 74, '：': 75, '它': 76, '粒': 77}


In [105]:
# 利用字典，映射數據
en_num_data = [[en2id[en] for en in line ] for line in en_data]
ch_num_data = [[ch2id[ch] for ch in line] for line in ch_data]
de_num_data = [[ch2id[ch] for ch in line][1:] for line in ch_data]

print('char:', en_data[1])
print('index:', en_num_data[1])

KeyError: ignored